Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.8344713415168299, θ=0.7872185614222056)   …  Distributions.Gamma(α=0.17612861488218834, θ=0.038251445779690085)
 Distributions.Gamma(α=0.2392558388843522, θ=0.1780994820783588)      Distributions.Gamma(α=0.3324554094065324, θ=0.6103039857465118)   
 Distributions.Gamma(α=0.820903968038649, θ=0.7725750492383641)       Distributions.Gamma(α=0.4914950079098066, θ=0.07179905186960855)  
 Distributions.Gamma(α=0.7081470910079337, θ=0.14603292428654635)     Distributions.Gamma(α=0.570065141009251, θ=0.46922207190131293)   
 Distributions.Gamma(α=0.8958490411186462, θ=0.9243424780572451)      Distributions.Gamma(α=0.7279441517685241, θ=0.9278618939898446)   )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.855353  0.0470957  0.397108  0.707681
 0.900955  0.164339   0.902798  0.653251
 0.167275  0.960696   0.380499  0.665275)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7733"),[:i,:j],3x4 Array{Float64,2}:
 0.489181  0.234274   0.96101   0.855737
 0.401874  0.103091   0.385555  0.65996 
 0.278337  0.0650815  0.379578  0.221928)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.8617423272717171, σ=0.926586714205176)    …  Distributions.Normal(μ=0.03503587212704251, σ=0.6525339370541043) 
 Distributions.Normal(μ=0.8227022478698605, σ=0.584724281858988)       Distributions.Normal(μ=0.9374612351685223, σ=0.24786663354613858) 
 Distributions.Normal(μ=0.02546694376610459, σ=0.9442752320371413)     Distributions.Normal(μ=0.24785199606269837, σ=0.09700803177543915)
 Distributions.Normal(μ=0.5414678592096374, σ=0.5333682639312844)      Distributions.Normal(μ=0.8834293005640899, σ=0.4851605458021737)  
 Distributions.Normal(μ=0.8486845272239543, σ=0.8616244550352423)      Distributions.Normal(μ=0.22477616748444418, σ=0.24887360569286465))

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.207816  0.427011  0.446451  0.789303
 0.745235  0.911809  0.967782  0.600531
 0.93621   0.276465  0.754275  0.229033)

# Make a factor

In [8]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;

# Make a (generated) value function

In [9]:
value(f)

-35.32039529167051

In [10]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [11]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [12]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [13]:
value(obs), value(μ_prior), value(τ_prior)

(-174.78259193013588,-13.593072740907873,0.0449816833123928)

In [14]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,[:i,:j],20x6 Array{Float64,2}:
 0.945483  0.0745032  0.155597  0.599954   0.601484   0.0594112
 0.414233  0.938643   0.853749  0.0607691  0.679546   0.846634 
 0.994395  0.19247    0.461456  0.986473   

In [15]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [16]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [17]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…

In [18]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (14.163555960376293,-11.000000000000002)
 (13.420224103504797,-11.000000000000002)
 (9.70449875156132,-11.000000000000002)  
 (9.834016297979943,-11.000000000000002) 
 (11.683802974512052,-11.000000000000002)
 (10.49440835323131,-11.000000000000002) 

In [19]:
naturals(obs, τ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,14.822260742742055)
 (10.0,14.725333092837271)
 (10.0,12.668761117329007)
 (10.0,12.887843428246299)
 (10.0,13.47665058594978) 
 (10.0,13.178482278567948)

In [20]:
check_conjugate(τ, m)

true

In [21]:
check_conjugate(μ, m)

true

In [22]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [23]:
update!(μ, Val{:conjugate})

LoadError: LoadError: UndefVarError: update! not defined
while loading In[23], in expression starting on line 1

In [24]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)